<a href="https://colab.research.google.com/github/yiyc-kr/llm-study/blob/main/llama_3_sqlcoder_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

vLLM으로 llama-3-sqlcoder-8b 한글과 프롬프트 변형(show create로 나오는 걸로) 테스트

In [1]:
# llama3를 지원하는 낮은 버전의 vllm 설치
# https://pypi.org/project/vllm/0.4.1/
!pip install vllm==0.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 21.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.8 MB/s 

In [4]:
# datasets 설치
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [1]:
import torch
# import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from vllm import LLM, SamplingParams

In [40]:
import sqlparse

In [2]:
# 구글드라이브 마운트된 경로
drive_path = 'drive/MyDrive/llm-study/'

In [3]:
# 테스트할 모델 경로
model_path = drive_path + "llama-3-sqlcoder-8b"

In [4]:
# 컨피그와 토크나이저 로드
config = AutoConfig.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
llm = LLM(
    model=model_path,
    # tokenizer_mode='auto',
    # gpu_memory_utilization=0.95,
    # tensor_parallel_size=2,
    # trust_remote_code=True,
    max_model_len=8192,
)

INFO 07-20 11:38:53 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='drive/MyDrive/llm-study/llama-3-sqlcoder-8b', speculative_config=None, tokenizer='drive/MyDrive/llm-study/llama-3-sqlcoder-8b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 07-20 11:38:54 utils.py:608] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 07-20 11:38:55 selector.py:77] Cannot use FlashAttention backend because the flash_attn package is not found. Please install it for better performance.
INFO 07-20 11:38:55 selector.py:33] Using XFormers backend.
INFO 07-20 11:43:18 model_runner.py:173] Loading model weights took 14.9595 GB
INFO 07-20 11:43:19 gpu_executor.py:119] # GPU blocks: 9558, # CPU blocks: 2048
INFO 07-20 11:43:22 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-20 11:43:22 model_runner.py:980] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
IN

In [16]:
sampling_params = SamplingParams(
            n=1,
            # best_of=None,
            # presence_penalty=0.0,
            # frequency_penalty=frequency_penalty,
            temperature=0.7,
            top_p=0.5,
            top_k=40,
            # max_tokens=max_tokens,
            max_tokens=10000,
            skip_special_tokens=True,
            # stop_token_ids=stop_token_ids,
            # num_return_sequences=1, # unexpected keyword argument
            # eos_token_id=tokenizer.eos_token_id,
            # pad_token_id=tokenizer.eos_token_id,
            # max_new_tokens=400,
            # do_sample=False,
            # num_beams=1
        )

In [6]:
# 테스트할 ddl이 삽입되어있는 프롬프트
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE corporate_customers (
    corporate_id INTEGER PRIMARY KEY, -- Unique ID for each corporate customer
    name VARCHAR(100) NOT NULL, -- Name of the corporate customer
    address VARCHAR(200), -- Mailing address of the corporate customer
    contact_person VARCHAR(100), -- Contact person in the corporate customer
    contact_phone VARCHAR(20), -- Contact phone number
    contact_email VARCHAR(100) -- Contact email address
);

CREATE TABLE branches (
    branch_id INTEGER PRIMARY KEY, -- Unique ID for each branch
    name VARCHAR(100) NOT NULL, -- Name of the branch
    address VARCHAR(200), -- Address of the branch
    phone VARCHAR(20), -- Contact phone number
    manager VARCHAR(100) -- Branch manager name
);

CREATE TABLE accounts (
    account_id INTEGER PRIMARY KEY, -- Unique ID for each account
    customer_id INTEGER, -- ID of the corporate customer owning the account
    branch_id INTEGER, -- ID of the branch where the account is held
    account_type VARCHAR(50), -- Type of the account (e.g., savings, checking)
    balance DECIMAL(15, 2), -- Current balance of the account
    opened_date DATE, -- Date when the account was opened
    FOREIGN KEY (customer_id) REFERENCES corporate_customers(corporate_id),
    FOREIGN KEY (branch_id) REFERENCES branches(branch_id)
);

CREATE TABLE products (
    product_id INTEGER PRIMARY KEY, -- Unique ID for each product
    name VARCHAR(100) NOT NULL, -- Name of the product
    description TEXT, -- Description of the product
    interest_rate DECIMAL(5, 2), -- Interest rate for the product
    created_date DATE -- Date when the product was created
);

CREATE TABLE customer_revenues (
    revenue_id INTEGER PRIMARY KEY, -- Unique ID for each revenue record
    customer_id INTEGER, -- ID of the corporate customer
    year INTEGER, -- Year of the revenue record
    total_revenue DECIMAL(15, 2), -- Total revenue for the year
    FOREIGN KEY (customer_id) REFERENCES corporate_customers(corporate_id)
);

CREATE TABLE account_transactions (
    transaction_id INTEGER PRIMARY KEY, -- Unique ID for each transaction
    account_id INTEGER, -- ID of the account involved in the transaction
    transaction_date DATE, -- Date of the transaction
    transaction_type VARCHAR(50), -- Type of the transaction (e.g., deposit, withdrawal)
    amount DECIMAL(15, 2), -- Amount of the transaction
    description TEXT, -- Description or memo for the transaction
    FOREIGN KEY (account_id) REFERENCES accounts(account_id)
);

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [59]:
# 테스트할 ddl이 삽입되어있는 프롬프트
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE `corporate_customers` (
  `corporate_id` NUMBER NOT NULL COMMENT '고객ID',
  `name` VARCHAR2(100) NOT NULL COMMENT '고객명',
  `address` VARCHAR2(200) COMMENT '주소',
  `contact_person` VARCHAR2(100) COMMENT '담당자',
  `contact_phone` VARCHAR2(20) COMMENT '전화번호',
  `contact_email` VARCHAR2(100) COMMENT '이메일',
  PRIMARY KEY (`corporate_id`)
) COMMENT='기업고객정보';

CREATE TABLE `branches` (
  `branch_id` NUMBER NOT NULL COMMENT '지점ID',
  `name` VARCHAR2(100) NOT NULL COMMENT '지점명',
  `address` VARCHAR2(200) COMMENT '주소',
  `phone` VARCHAR2(20) COMMENT '전화번호',
  `manager` VARCHAR2(100) COMMENT '지점장',
  PRIMARY KEY (`branch_id`)
) COMMENT='지점정보';

CREATE TABLE `accounts` (
  `account_id` NUMBER NOT NULL COMMENT '계좌ID',
  `customer_id` NUMBER COMMENT '고객ID',
  `branch_id` NUMBER COMMENT '지점ID',
  `account_type` VARCHAR2(50) COMMENT '계좌유형',
  `balance` NUMBER(15, 2) COMMENT '잔액',
  `opened_date` DATE COMMENT '개설일',
  PRIMARY KEY (`account_id`),
  FOREIGN KEY (`customer_id`) REFERENCES `corporate_customers`(`corporate_id`),
  FOREIGN KEY (`branch_id`) REFERENCES `branches`(`branch_id`)
) COMMENT='계좌정보';

CREATE TABLE `products` (
  `product_id` NUMBER NOT NULL COMMENT '상품ID',
  `name` VARCHAR2(100) NOT NULL COMMENT '상품명',
  `description` CLOB COMMENT '설명',
  `interest_rate` NUMBER(5, 2) COMMENT '이자율',
  `created_date` DATE COMMENT '생성일',
  PRIMARY KEY (`product_id`)
) COMMENT='상품정보';

CREATE TABLE `customer_revenues` (
  `revenue_id` NUMBER NOT NULL COMMENT '매출ID',
  `customer_id` NUMBER COMMENT '고객ID',
  `year` NUMBER COMMENT '연도',
  `total_revenue` NUMBER(15, 2) COMMENT '총매출',
  PRIMARY KEY (`revenue_id`),
  FOREIGN KEY (`customer_id`) REFERENCES `corporate_customers`(`corporate_id`)
) COMMENT='고객매출정보';

CREATE TABLE `account_transactions` (
  `transaction_id` NUMBER NOT NULL COMMENT '거래ID',
  `account_id` NUMBER COMMENT '계좌ID',
  `transaction_date` DATE COMMENT '거래일',
  `transaction_type` VARCHAR2(50) COMMENT '거래유형',
  `amount` NUMBER(15, 2) COMMENT '거래금액',
  `description` CLOB COMMENT '설명',
  PRIMARY KEY (`transaction_id`),
  FOREIGN KEY (`account_id`) REFERENCES `accounts`(`account_id`)
) COMMENT='거래기록';


### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

In [65]:
# 질문
question = "각 지점별로 거래량이 많은 고객들의 이름을 알려줘 그리고 각 고객은 얼마가 있어?"

In [66]:
updated_prompt = prompt.format(question=question)

In [67]:
output = llm.generate(updated_prompt, sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.30s/it]


In [68]:
print(sqlparse.format(output[0].outputs[0].text, reindent=True))

SELECT b.name AS branch_name,
       c.name AS customer_name,
       SUM(at.amount) AS total_transaction_amount
FROM account_transactions AT
JOIN accounts a ON at.account_id = a.account_id
JOIN branches b ON a.branch_id = b.branch_id
JOIN corporate_customers c ON a.customer_id = c.corporate_id
GROUP BY b.name,
         c.name
ORDER BY total_transaction_amount DESC NULLS LAST;


인풋 토큰 분석

In [58]:
# 텍스트를 토큰화
tokens = tokenizer(updated_prompt, return_tensors="pt")

# 토큰 아이디와 실제 토큰 출력
input_ids = tokens["input_ids"].squeeze().tolist()
# tokens_with_values = tokenizer.convert_ids_to_tokens(input_ids)

# 스페셜 토큰 확인
special_tokens = tokenizer.all_special_tokens
special_token_ids = tokenizer.all_special_ids

# 결과 출력
print("Tokens with values:")
for token_id in input_ids:
    print(f"Token ID: {token_id}, Token Value: {repr(tokenizer.decode(token_id))}")

print("\nSpecial Tokens:")
for token, token_id in zip(special_tokens, special_token_ids):
    print(f"Special Token: {token}, Token ID: {token_id}")

Tokens with values:
Token ID: 128000, Token Value: '<|begin_of_text|>'
Token ID: 14711, Token Value: '###'
Token ID: 5546, Token Value: ' Task'
Token ID: 198, Token Value: '\n'
Token ID: 32215, Token Value: 'Generate'
Token ID: 264, Token Value: ' a'
Token ID: 8029, Token Value: ' SQL'
Token ID: 3319, Token Value: ' query'
Token ID: 311, Token Value: ' to'
Token ID: 4320, Token Value: ' answer'
Token ID: 510, Token Value: ' ['
Token ID: 53528, Token Value: 'QUESTION'
Token ID: 60, Token Value: ']'
Token ID: 101930, Token Value: '각'
Token ID: 67890, Token Value: ' 지'
Token ID: 101838, Token Value: '점'
Token ID: 57575, Token Value: '에서'
Token ID: 107120, Token Value: ' 가장'
Token ID: 115983, Token Value: ' 돈'
Token ID: 13094, Token Value: '이'
Token ID: 110187, Token Value: ' 많은'
Token ID: 116534, Token Value: ' 고객'
Token ID: 13094, Token Value: '이'
Token ID: 123621, Token Value: ' 누구'
Token ID: 101203, Token Value: '며'
Token ID: 95303, Token Value: ' 계'
Token ID: 113002, Token Value: '좌'


In [46]:
len(input_ids)

686

In [54]:
tokenizer.decode(118176)

' 얼마'

In [48]:
tokenizer.convert_ids_to_tokens([118176])

['Ġìĸ¼ë§Ī']

In [ ]:
# 한국어 토크나이저 연구용
# https://huggingface.co/saltlux/Ko-Llama3-Luxia-8B